In [1]:
#Quantifying the probability of PopIII remnant growth
import matplotlib
matplotlib.use('Agg')
import yt
import numpy as np
import sys
from scipy.signal import savgol_filter
import matplotlib.pyplot as plt
import glob
from yt.utilities.physical_constants import mh, mp, me, G, kboltz, pi
from math import pi
from yt.units.yt_array import YTQuantity, YTArray





In [2]:
def BondiRadius(M_BH, cs, v):
    R = G * M_BH / (cs*cs + v*v)
    return R


In [3]:
#Set up out systems
R_coregal = YTQuantity(20, 'pc') #Setting the radius for the random walk at 20 pc (conservative)
R_cloud = YTQuantity(0.1, 'pc')  #difficult to quantify. Conservative?
#N_clouds = 1000                   #Hard to see how it is larger than this. Conservative?
M_BH = YTQuantity(500, 'msun')   #optimistic
Temp = YTQuantity(10000, 'K')    #can vary (lower) this but 10000 K is reasonable
Gamma = 5.0/3.0
Mu = 1.22
v_bh = YTQuantity(10, 'km/s')     #reasonable compared to circular velocity of these haloes. 
tau_hubble = YTQuantity(13.8, 'Gyr') #age of Universe
f = 1e-4                          # fraction of volume filled with dense gas





N_clouds = f * np.power(R_coregal/R_cloud, 3.0)
soundspeed2 = Gamma*kboltz*Temp/(Mu*mh)
print("soundspeed2 = ", soundspeed2)
soundspeed = YTQuantity(np.sqrt(soundspeed2).d, 'cm/s')
cs = soundspeed
print("cs = ", cs.to('km/s'))
print("Black Hole Mass = ", M_BH)
R_bondi = BondiRadius(M_BH.in_units('g'), cs, v_bh)
print("R_bondi = %e pc\t %e au" % (R_bondi.to('pc'), R_bondi.to('au')))
print("N_clouds = ", N_clouds)


v_bh = cs

soundspeed2 =  1126900009996.2607 erg/g
cs =  10.615554672254584 km/s
Black Hole Mass =  500 Msun
R_bondi = 1.011010e-02 pc	 2.085358e+03 au
N_clouds =  800.0 dimensionless


In [4]:
#We assume that the Black Hole (defined by it's Bondi Hoyle radius) makes a random walk around the galaxy. 
#To calculate the probability of a black hole growing we calculate the probability that during that random 
#walk that the black hole intersects a gas cloud

# P_growth = Probability of black hole being in gas cloud (P_cloud) * Number of clouds (N_clouds) * Number of iterations (N_iters)

P_cloud = np.power(R_cloud/R_coregal, 3.0)

#Number of iterations is equivalent to distance travelled in a given time

#Time to sample the galactic core = (R_core/R_bondi)^3 * (2 * R_bondi/v_bh) == tau_sample

#Then the fraction sampled = tau_hubble/tau_sampled

tau_sample = np.power(R_coregal/R_bondi, 3.0) * (2 * R_bondi/v_bh)

print("tau_sample = ", tau_sample.to('Gyr'))

P_sampled = tau_hubble/tau_sample
print("Percentage of galaxy sampled = %1.6f percent" % (100*tau_hubble/tau_sample))


P_growth = P_cloud * N_clouds * P_sampled

print("Probability of Black Holes encountering a gas cloud = %1.2e" % (P_growth))

tau_sample =  14418.267524422059 Gyr
Percentage of galaxy sampled = 0.095712 percent
Probability of Black Holes encountering a gas cloud = 9.57e-08
